In [ ]:
# mLife Data Parser - Beispiel & Testdokumentation

Dieses Notebook zeigt die grundlegende Nutzung des mLife Data Parsers.

## Übersicht

Der Parser konvertiert komplexe mLife-CSV-Exporte in ein einheitliches Long Format mit folgenden Spalten:
- `timestamp`: Zeitpunkt der Messung
- `source_type`: Datenquelle (Vitals, Lab, Medication, etc.)
- `category`: Unterkategorie
- `parameter`: Parametername
- `value`: Messwert

In [ ]:
# Imports
from mlife_core.services.pipeline import run_parsing_pipeline
from mlife_core.services.parsers import DataParser
import pandas as pd

# Beispieldatei
INPUT_FILE = "data/gesamte_akte2.csv"

## 1. Schnellstart: Komplette Pipeline

Die einfachste Nutzung - alle Daten werden geparst und konsolidiert:

In [ ]:
# Komplette Pipeline ausführen
df = run_parsing_pipeline(INPUT_FILE)

print(f"Geladene Datenpunkte: {len(df):,}")
print(f"Zeitraum: {df['timestamp'].min()} bis {df['timestamp'].max()}")
print(f"Gefundene Quelltypen: {df['source_type'].nunique()}")
df.head(10)

## 2. Übersicht der Datenstruktur

Welche Datentypen wurden gefunden?

In [ ]:
# Übersicht nach source_type
print("=== Datenpunkte pro Quelltyp ===")
source_counts = df.groupby('source_type').size().sort_values(ascending=False)
print(source_counts.to_string())

print("\n=== Alle eindeutigen Quelltypen ===")
for src in sorted(df['source_type'].unique()):
    print(f"  - {src}")

## 3. Einzelne Parser direkt nutzen

Für feinere Kontrolle kann der `DataParser` direkt genutzt werden:

In [ ]:
# Parser initialisieren
parser = DataParser(INPUT_FILE, delimiter=";")

# Nur Vitaldaten parsen
vitals = parser.parse_vitals()
print(f"Vitaldaten: {len(vitals)} Einträge")
print(f"Parameter: {vitals['parameter'].unique()[:10]}...")  # Erste 10

vitals.head()

In [ ]:
# Labordaten parsen
lab = parser.parse_lab()
print(f"Labordaten: {len(lab)} Einträge")
print(f"Kategorien: {lab['category'].unique()}")

lab.head()

In [ ]:
# Medikamentendaten parsen
meds = parser.parse_medication_logic()
print(f"Medikamentendaten: {len(meds)} Einträge")
print(f"Medikamente: {meds['parameter'].unique()[:10]}...")

meds.head()

## 4. Filtern nach spezifischen Daten

Beispiele für typische Filteroperationen:

In [ ]:
# Alle ECMO-Daten finden (in source_type, category oder parameter)
ecmo_mask = (
    df['source_type'].astype(str).str.contains('ECMO', case=False) |
    df['category'].astype(str).str.contains('ECMO', case=False) |
    df['parameter'].astype(str).str.contains('ECMO', case=False)
)
ecmo_data = df[ecmo_mask]
print(f"ECMO-Datenpunkte: {len(ecmo_data)}")
ecmo_data.head()

In [ ]:
# Nur Herzfrequenz-Daten
hf_data = df[df['parameter'].str.contains('HF|Herzfrequenz', case=False, na=False)]
print(f"Herzfrequenz-Datenpunkte: {len(hf_data)}")
hf_data.head()

## 5. Export

Daten können einfach als CSV oder Excel exportiert werden:

In [ ]:
from mlife_core.utils.export import save_dataframe, get_subset_df

# Komplettexport
# save_dataframe(df, "export_komplett.csv")

# Nur Vitaldaten exportieren
vitals_subset = get_subset_df(df, "vitals")
print(f"Vitaldaten zum Export: {len(vitals_subset)} Zeilen")
# save_dataframe(vitals_subset, "export_vitals.csv")

# Verfügbare Subset-Keys:
print("\nVerfügbare Subset-Keys für get_subset_df():")
print("  - vitals, lab, respiratory, medication")
print("  - impella, crrt, ecmo, nirs, doctor_notes")

## 6. Verfügbare Parser-Methoden

Übersicht aller verfügbaren Parsing-Methoden des `DataParser`:

In [ ]:
# Alle öffentlichen Methoden des DataParsers
public_methods = [m for m in dir(parser) if not m.startswith('_') and callable(getattr(parser, m))]
print("Verfügbare Parser-Methoden:")
for method in public_methods:
    print(f"  - parser.{method}()")